In [11]:
%load_ext autoreload
%autoreload 2
%autosave 60

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 60 seconds


In [12]:
import os

os.chdir("/usr/src/app")

In [13]:
import requests

import polars as pl

dataset = 'https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/global-shark-attack/exports/csv?delimiter=%3B'

df = pl.read_csv(requests.get(dataset).content, separator=';')
df.head()

date,year,type,country,area,location,activity,name,sex,age,injury,fatal_y_n,time,species,investigator_or_source,pdf,href_formula,href,case_number,case_number0,original_order
str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64
"""2023-07-04""",2023,"""Unprovoked""","""USA""","""New York""","""Cherry Grove, …","""Standing""","""female""","""F""",null,"""Puncture wound…","""N""","""16h25""","""4' shark""","""S.S. Curatolo-…",null,null,null,null,null,null
null,2023,"""Unprovoked""","""USA""","""Florida""","""Outer Mole, Fo…","""Swimming""","""Nicolae Cioban…","""M""","""34""","""Lacerations to…","""N""","""14h00""",null,"""Miami Herald, …",null,null,null,null,null,null
"""2022-07-19""",2022,"""Questionable""","""USA""","""Florida""","""Cape Canaveral…",null,null,null,null,"""minor injuries…","""N""","""10jh45""","""Shark involvem…","""WESH, 7/20/202…","""2022.07.19.b-C…","""http://sharkat…","""http://sharkat…","""2022.07.19.b""","""2022.07.19.b""",6788
"""2022-02-05""",2022,"""Unprovoked""","""AUSTRALIA""","""Western Austra…","""Beds, Wylie Ba…","""Floating in in…","""Jacquelin Morl…","""F""","""20.0""","""Torso bitten""","""N""","""11h15""","""White shark""","""K. McMurray, T…","""2022.02.05-Mor…","""http://sharkat…","""http://sharkat…","""2022.02.05""","""2022.02.05""",6734
"""2022-01-09""",2022,"""Unprovoked""","""NEW ZEALAND""","""South Island""","""Friendly Bay, …","""Swimming""","""Alvira Repia-K…","""F""","""13.0""","""Puncture wound…","""N""","""16h30""","""Sevengill shar…","""Otago Daily Ti…","""2022.01.09-Rep…","""http://sharkat…","""http://sharkat…","""2022.01.09""","""2022.01.09""",6730


In [14]:
from app.shared.constants import CountryMap, StateMap

attack_df = (
    df
    .with_columns(
        country=pl.col("country").str.to_lowercase().map_dict(CountryMap.reverse_records),
        state=pl.col("area").str.to_lowercase().map_dict(StateMap.reverse_records),
    )
    .drop("area")
    .drop_nulls(["country", "state"])
)
attack_df.head()

date,year,type,country,location,activity,name,sex,age,injury,fatal_y_n,time,species,investigator_or_source,pdf,href_formula,href,case_number,case_number0,original_order,state
str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str
"""2008-06-01""",2008,"""Unprovoked""","""BR""","""Piedade, Recif…","""Swimming""","""Wellington dos…","""M""","""14""","""Hand severed, …","""N""","""15h00""","""Bull shark""","""Associated Pre…","""2008.06.01-San…","""http://sharkat…","""http://sharkat…","""2008.06.01.a""","""2008.06.01.a""",5075,"""PE"""
"""1997-07-02""",1997,"""Unprovoked""","""BR""","""Paiva""","""Surfing""","""Jurandir Amori…",null,null,"""Right thigh bi…","""N""",null,null,"""D. Duarte; Glo…","""1997.07.02-Jur…","""http://sharkat…","""http://sharkat…","""1997.07.02.R""","""1997.07.02.R""",4081,"""PE"""
"""1996-10-28""",1996,"""Unprovoked""","""BR""","""Barra de Janga…","""Surfing""","""Luis Henrique …","""M""",null,"""Survived""","""N""",null,null,"""JCOnline""","""1996.10.28.a-M…","""http://sharkat…","""http://sharkat…","""1996.10.28.a""","""1996.10.28.a""",4053,"""PE"""
"""2009-09-07""",2009,"""Unprovoked""","""BR""","""Piedade, Recif…","""Swimming, atte…","""Geovanni Tiago…","""M""","""15""","""FATAL""","""Y""","""Afternoon""",null,"""C. Eksander, G…","""2009.09.07-Bar…","""http://sharkat…","""http://sharkat…","""2009.09.07""","""2009.09.07""",5236,"""PE"""
"""2018-06-03""",2018,"""Unprovoked""","""BR""","""Piedade Beach,…","""Swimming""","""Jose Ernesto d…","""M""","""18.0""","""FATAL""","""Y""","""Late afternoon…","""Tiger shark""","""Diario de Pern…","""2018.06.03.a-d…","""http://sharkat…","""http://sharkat…","""2018.06.03.a""","""2018.06.03.a""",6328,"""PE"""


In [15]:
from app.core.models import Beach

data = Beach.objects.values()
beach_df = pl.DataFrame(list(data))
beach_df.head()

id,city,state,country,name,latitude,longitude
i64,str,str,str,str,f64,f64
1,"""Goiana""","""PE""","""BR""","""Carne de Vaca""",0.0,0.0
2,"""Goiana""","""PE""","""BR""","""Pontas de Pedr…",0.0,0.0
3,"""Goiana""","""PE""","""BR""","""Barra de Catua…",0.0,0.0
4,"""Goiana""","""PE""","""BR""","""catuama""",0.0,0.0
5,"""Goiana""","""PE""","""BR""","""Atapuz""",0.0,0.0


In [16]:
tmp_folder = "."

atk_parquet = f"{tmp_folder}/attacks.parquet"
bch_parquet = f"{tmp_folder}/beaches.parquet"

attack_df.write_parquet(f"{tmp_folder}/attacks.parquet", use_pyarrow=True)
beach_df.write_parquet(f"{tmp_folder}/beaches.parquet", use_pyarrow=True)

In [17]:
import duckdb

con = duckdb.connect()

In [28]:
data = con.execute(f"""
    with distances as (
        select
            bch.id as bch_id,
            atk.original_order as atk_id,
            bch.country as country,
            bch.state as state,
            regexp_replace(lower(bch.name), 'praia|beach', '', 'g') as beach_name,
            regexp_replace(lower(atk.location), 'praia|beach', '', 'g') as atk_location,
            regexp_replace(str_split(lower(atk.location), ',')[1], 'praia|beach|(\s\s)', '', 'g') as seg_atk_location,
            levenshtein(beach_name, atk_location) as distance,
            levenshtein(beach_name, seg_atk_location) as seg_distance,
            least(distance, seg_distance) as smaller_distance,
            min(smaller_distance) over (partition by atk_id) as min_distance,
            first(bch_id) over (partition by atk_id order by smaller_distance) as closest
        from read_parquet("{bch_parquet}") bch
        join read_parquet("{atk_parquet}") atk
            on bch.country = atk.country and bch.state = atk.state
    ),
    closests as (
        select
            d.atk_id,
            first(d.min_distance) as min_distance,
            first(d.closest) as bch_id,
        from distances d
        group by 1
    )
    select
        bch.id as beach_id,
        atk.location,
        bch.name,
        atk.date,
    from closests c
    join read_parquet("{bch_parquet}") bch on bch.id = c.bch_id
    join read_parquet("{atk_parquet}") atk on atk.original_order = c.atk_id
    order by c.min_distance desc
""").fetchall()

data

[(57, 'Fernando de Noronha Marine Park', 'Barra de Sirinhaem', '2017-02-06'),
 (3, 'Body recovered at Goiana', 'Barra de Catuama', '2006-07-10'),
 (15, 'Fernano de Noronha', 'Forno de Cal', '2015-12-21'),
 (33, 'Punta Del Chifre Beach, Olinda', 'Del Chifre', '2011-11-12'),
 (27, 'Coral Cove Beach', 'Rio Doce', '2013-05-23'),
 (33, 'Punta Del Chifre Beach, Olinda', 'Del Chifre', '2006-06-18'),
 (33, 'Punta Del Chifre, Olinda', 'Del Chifre', '2008-06-11'),
 (27, 'Coral Cove, Cabo de Santo Agostinho', 'Rio Doce', '2012-08-26'),
 (62, 'Itamaracá', 'Tamandaré', '2007-12-07'),
 (35, 'Praia do Pina', 'Pina', '2011-07-31'),
 (35, 'Praia do Pina', 'Pina', '2011-06-29'),
 (24, 'Praia de Pau Amarelo, Recife', 'Pau Amarelo', '1991-04-24'),
 (37,
  'Piedade Beach, Jaboatão dos Guararapes City, Recife',
  'Piedade',
  '2002-10-14'),
 (37, 'Piedade Beach, Recife', 'Piedade', '2004-05-22'),
 (72, 'Cacimba do Padre Beach', 'Cacimba do Padre', '2021-01-13'),
 (37, 'Piedade Beach', 'Piedade', '2004-05-23

In [19]:
fatal_map = {
    "Y": True,
    "N": False,
}

data.with_columns(
    pl.col("fatal_y_n").map_dict(fatal_map, default=None).alias("fatal"), 
    pl.col("age").cast(pl.Float32, strict=False).alias("age")
).head()

beach_id,date,year,type,country,location,activity,name,sex,age,injury,fatal_y_n,time,species,investigator_or_source,pdf,href_formula,href,case_number,case_number0,original_order,state,fatal
i64,str,i64,str,str,str,str,str,str,f32,str,str,str,str,str,str,str,str,str,str,i64,str,bool
57,"""2017-02-06""",2017,"""Provoked""","""BR""","""Fernando de No…","""Grabbing shark…","""female""","""F""",null,"""Minor injury t…","""N""","""Afternoon""","""Lemon shark pu…","""O Popular, 2/7…","""2017.02.06.b-B…","""http://sharkat…","""http://sharkat…","""2017.02.06.b""","""2017.02.06.b""",6152,"""PE""",false
3,"""2006-07-10""",2006,"""Unprovoked""","""BR""","""Body recovered…",null,"""Unidentified""","""M""",null,"""FATAL""","""Y""",null,"""Bull or tiger …","""Qualittas; F. …","""2006.07.10-Goi…","""http://sharkat…","""http://sharkat…","""2006.07.10""","""2006.07.10""",4867,"""PE""",true
15,"""2015-12-21""",2015,"""Unprovoked""","""BR""","""Fernano de Nor…","""Scuba diving""","""Márcio de Cast…","""M""",32.0,"""Right hand & p…","""N""",null,"""Tiger shark, 1…","""Fox News, 12/2…","""2015.12.21.a-B…","""http://sharkat…","""http://sharkat…","""2015.12.21.a""","""2015.12.21.a""",6004,"""PE""",false
33,"""2011-11-12""",2011,"""Unprovoked""","""BR""","""Punta Del Chif…","""Surfing""","""Jerônimo Perei…","""M""",35.0,"""Legs bitten""","""N""",null,null,"""H. Nickel & A.…","""2011.11.12-Per…","""http://sharkat…","""http://sharkat…","""2011.11.12""","""2011.11.12""",5483,"""PE""",false
27,"""2013-05-23""",2013,"""Unprovoked""","""BR""","""Coral Cove Bea…",null,"""José Rogério d…","""M""",41.0,"""FATAL""","""Y""",null,null,"""JC Online, 6/6…","""2013.05.25.b-d…","""http://sharkat…","""http://sharkat…","""2013.05.23.b""","""2013.05.23.b""",5655,"""PE""",true


In [20]:
from app.core.models import SharkAttack

fatal_map = {
    "Y": True,
    "N": False,
}

processed_data = data.with_columns(
    pl.col("fatal_y_n").map_dict(fatal_map, default=None).alias("fatal"),  
    pl.col("age").cast(pl.Float32, strict=False).alias("age")
)

attacks = []

for atk in processed_data.to_dicts():
    atk = SharkAttack(
        beach_id=atk["beach_id"],
        date=atk["date"],
        type=atk["type"],
        country=atk["country"],
        state=atk["state"],
        location=atk["location"],
        activity=atk["activity"],
        victim_name=atk["name"],
        victim_gender=atk["sex"],
        victim_age=atk["age"],
        victim_injury=atk["injury"],
        fatal=atk["fatal"],
        time=atk["time"],
        species=atk["species"],
        source=atk["investigator_or_source"],
    )
    attacks.append(atk)

SharkAttack.objects.bulk_create(attacks)

[<SharkAttack: SharkAttack object (1)>,
 <SharkAttack: SharkAttack object (2)>,
 <SharkAttack: SharkAttack object (3)>,
 <SharkAttack: SharkAttack object (4)>,
 <SharkAttack: SharkAttack object (5)>,
 <SharkAttack: SharkAttack object (6)>,
 <SharkAttack: SharkAttack object (7)>,
 <SharkAttack: SharkAttack object (8)>,
 <SharkAttack: SharkAttack object (9)>,
 <SharkAttack: SharkAttack object (10)>,
 <SharkAttack: SharkAttack object (11)>,
 <SharkAttack: SharkAttack object (12)>,
 <SharkAttack: SharkAttack object (13)>,
 <SharkAttack: SharkAttack object (14)>,
 <SharkAttack: SharkAttack object (15)>,
 <SharkAttack: SharkAttack object (16)>,
 <SharkAttack: SharkAttack object (17)>,
 <SharkAttack: SharkAttack object (18)>,
 <SharkAttack: SharkAttack object (19)>,
 <SharkAttack: SharkAttack object (20)>,
 <SharkAttack: SharkAttack object (21)>,
 <SharkAttack: SharkAttack object (22)>,
 <SharkAttack: SharkAttack object (23)>,
 <SharkAttack: SharkAttack object (24)>,
 <SharkAttack: SharkAttac